In [1]:
import numpy as np
import  functools
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict, Counter

In [2]:
# 汉字字体，优先使用楷体，找不到则使用黑体
plt.rcParams['font.sans-serif'] = ['Kaitt', 'SimHei']
 
# 正常显示负号
plt.rcParams['axes.unicode_minus'] = False

In [3]:
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('expand_frame_repr', False)  # 即“禁止换行”
# pd.set_option('display.precision', 2) #展示两位小数点
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
df_data_交易=pd.read_csv('data/账户交易信息.csv')
df_data_交易.columns = ['交易流水序号','账户代号','对方账号','借贷标志','交易金额','交易余额','对方行号','交易日期','交易时间','交易渠道','摘要代号','对方名称长度']
df_data_交易 = df_data_交易.sort_values(["交易日期" , "交易时间"])
df_data_账户=pd.read_csv('data/账户静态信息.csv')
df_data_账户.columns = ['账户代号','开户日期','开户行代号','客户性别','年龄']
df_label_train=pd.read_csv('data/训练集标签.csv')
df_label_train.columns = ['账户代号','label']
df_label_test=pd.read_csv('data/test_dataset.csv')
df_label_test.columns = ['账户代号']## 读取数据
df_label_all=pd.concat([df_label_train,df_label_test])

In [5]:
list_交易_金额为负_index = list(df_data_交易[df_data_交易["交易金额"] < 0].index)
list_mz_index = [] ;
list_交易_金额为负_对应的失败交易_index = [] ;
for i in list_交易_金额为负_index :
    S_now = df_data_交易.loc[i] ;
    j = i ;
    while (True) :
        j -= 1 ;
        S_nex = df_data_交易.loc[j] ;
        if (S_now["交易金额"] == -S_nex["交易金额"]) :
            if (S_now["账户代号"] == S_nex["账户代号"]) :
                list_交易_金额为负_对应的失败交易_index.append(j) ;
                list_mz_index.append(i) ;
                break ;
list_交易_抹账需要去除_index = list_交易_金额为负_index + list_交易_金额为负_对应的失败交易_index ;
df_data_交易_去除抹账后=df_data_交易[~df_data_交易.index.isin(list_交易_抹账需要去除_index)]

In [6]:
df_data_交易_去除抹账后 = df_data_交易_去除抹账后.sort_values(["交易日期" , "交易时间"])

In [57]:
# 找到每个客户最后一笔交易日期
df_tz_last_and_biggist = df_label_all[["账户代号"]].copy()

In [58]:
df_tz_last_and_biggist.head()

,账户代号
0,2029FF26D4E2CA79
1,09F5B90D46FB7CCE
2,E0C880EB18F4EFE8
3,ED8FFDCCF93C2F11
4,8D2F48B0BFC40AB2


In [ ]:
df_tmp = pd.DataFrame(data=None,columns=['账户代号','最大交易金额发生日期'])
for (name, df) in df_data_交易_去除抹账后.groupby("账户代号") :
    df_tmp = df_tmp.append({'账户代号': name , "最大交易金额发生日期" : pd.to_datetime(df.nlargest(1 , "交易金额")["交易日期"]) }, ignore_index=True)

In [88]:
df_tz_last_and_biggist = df_tz_last_and_biggist.merge(df_tmp , on="账户代号" , how="left")

In [89]:
df_tz_last_and_biggist.head()

,账户代号,最大交易金额发生日期
0,2029FF26D4E2CA79,"347963 2020-04-08 Name: 交易日期, dtype: datetim..."
1,09F5B90D46FB7CCE,"473507 2020-04-23 Name: 交易日期, dtype: datetim..."
2,E0C880EB18F4EFE8,"631777 2020-05-11 Name: 交易日期, dtype: datetim..."
3,ED8FFDCCF93C2F11,"569665 2020-05-04 Name: 交易日期, dtype: datetim..."
4,8D2F48B0BFC40AB2,"738437 2020-05-23 Name: 交易日期, dtype: datetim..."


In [98]:
type (df_tz_last_and_biggist.iloc[1 , 1])
print (df_tz_last_and_biggist.iloc[1 , 1])

473507   2020-04-23
Name: 交易日期, dtype: datetime64[ns]


In [94]:
df_tmp = df_data_交易_去除抹账后.groupby("账户代号")["交易日期"].max().reset_index().head()
df_tmp["交易日期"] = pd.to_datetime(df_tmp["交易日期"])
df_tmp.columns = ["账户代号" , "最后交易日期"]

In [95]:
df_tmp.head()

,账户代号,最后交易日期
0,00037295453A928A,2020-05-31
1,0004CC075464D54B,2020-05-21
2,000AA77144DC1BCC,2020-05-31
3,001ABBF3373AFC5B,2020-05-24
4,00310769938BC172,2020-05-29


In [100]:
type (df_tmp.iloc[1 , 1])

pandas._libs.tslibs.timestamps.Timestamp

In [75]:
import pandas as pd
df = pd.DataFrame(data=None,columns=['时间','车牌','北纬','东经'])
df

,时间,车牌,北纬,东经
